##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# 使用 Estimators 构建一个线性模型

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.google.cn/tutorials/estimator/linear"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" />在 Tensorflow.org 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/zh-cn/tutorials/estimator/linear.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" />在 Google Colab 运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/zh-cn/tutorials/estimator/linear.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" />在 Github 上查看源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/zh-cn/tutorials/estimator/linear.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png" />下载此 notebook</a>
  </td>
</table>

Note: 我们的 TensorFlow 社区翻译了这些文档。因为社区翻译是尽力而为， 所以无法保证它们是最准确的，并且反映了最新的
[官方英文文档](https://www.tensorflow.org/?hl=en)。如果您有改进此翻译的建议， 请提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 仓库。要志愿地撰写或者审核译文，请加入
[docs-zh-cn@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-zh-cn)。

## 概览

这份端到端演示使用 `tf.estimator` API 训练了一个逻辑回归模型。这个模型常被用作其他更复杂的算法的基线。

## 设置

In [0]:
!pip install sklearn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

## 加载泰坦尼克号数据集
你将使用泰坦尼克数据集，其中包含乘客的性别、年龄、船舱等级等属性，（相当病态的）目标是预测乘客的存活几率。

In [0]:
!pip install tensorflow==2.0.0-beta1
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [0]:
# 加载数据集。
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

## 探索数据

数据集包含以下特征

In [0]:
dftrain.head()

In [0]:
dftrain.describe()

在训练集和验证集中分别有 627 和 264 个样本

In [0]:
dftrain.shape[0], dfeval.shape[0]

大部分乘客是 20 多岁和 30 多岁。

In [0]:
dftrain.age.hist(bins=20)

船上的男性乘客数量大概是女性乘客的两倍。

In [0]:
dftrain.sex.value_counts().plot(kind='barh')

大部分乘客在“三等”舱。

In [0]:
dftrain['class'].value_counts().plot(kind='barh')

女性的存活几率远大于男性。对模型来说这显然是一个可预测的特征。

In [0]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

## 模型的特征工程
Estimators 使用一个名叫 [feature columns（特征列）](https://tensorflow.google.cn/guide/feature_columns) 的系统来描述模型应该如何解读每一个原始输入特征。一个 Estimator 期望一个数字输入向量，*特征列* 描述模型应该如何转换每个特征。

要学习一个有效的模型，关键是选择和制作一组正确的特征列。一个特征列可能是初始特征 `dict` 中的一组原始输入（一个*基本特征列*），或者是使用基于一个或多个基本列的变换而创建的任何新列（一组*派生特征列*）。

线性 estimator 使用数字型特征和类别型特征。特征列可用于所有 TensorFlow estimator，它们的目的是定义用于模型的特征列。并且，它们提供一些特征工程能力，比如 one-hot-encoding（独热编码），normalization（归一化）和 bucketization（分段）。

### 基本特征列

In [0]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

`input_function` 指定数据如何被转换为一个 `tf.data.Dataset`，该数据集以流的方式投喂输入管道。`tf.data.Dataset` 接受多个来源，如数据帧、csv格式文件等等。

In [0]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

你可以检查数据集：

In [0]:
ds = make_input_fn(dftrain, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['class'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

你也可以使用 `tf.keras.layers.DenseFeatures` 层检查特定特征列的结果：

In [0]:
age_column = feature_columns[7]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()

`DenseFeatures` 只接受密集 tensor（张量），要检查一个类别列你需要先将该列变换为一个指标列：

In [0]:
gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()

将所有基本特征添加到模型之后，让我们来训练模型吧。使用 `tf.estimator` API 训练一个模型只需要一个命令：

In [0]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

### 派生特征列

现在你达到了 75% 的准确率。要解释数据，分别使用每一个基本特征列可能不够。例如，性别和标签的相关性可能在不同年龄中有所差异。因此，针对 `gender="Male"` 和 `gender="Female"` 如果你只学习一个模型权重，你就不会抓住每一个“年龄-性别”组合（如：区分 `gender="Male"` - `age="30"` 和 `gender="Male"` - `age="40"`）。

要学习不同特征组合间的区别，你可以添加*交叉特征列*来建模（你也可以在交叉列之前分段年龄列）：

In [0]:
age_x_gender = tf.feature_column.crossed_column(['age', 'sex'], hash_bucket_size=100)

在模型中加入组合特征后，我们再来训练模型：

In [0]:
derived_feature_columns = [age_x_gender]
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns+derived_feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

现在它达到了 77.6% 的准确率，这比只用基本列训练要稍好一些。你可以尝试使用更多的特征和变换来看看你能否做得更好！

现在你可以用这个训练过的模型来预测验证集中的一名乘客。TensorFlow 模型已被优化，能同时对一个示例 batch 或者集合进行预测。之前的 `eval_input_fn` 是用这个验证集定义的。

In [0]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

最后，看看结果的接收者操作特征（ROC），它会使我们更好的了解真阳性率和假阳性率之间的权衡。

In [0]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)